In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# parquetファイル準備
df = pd.read_csv(r'input\user.csv')
table = pa.Table.from_pandas(df)
pq.write_table(table, r'input\user.parquet')

In [2]:
# 準備したparquetを読み込み
users = spark.read.parquet(r'input\user.parquet')

In [3]:
# 性別でパーティション分けして出力
users.where(users.sex == '男') \
     .write \
     .parquet(r'users/gnd=male'
              , mode='overwrite')

In [4]:
users.where(users.sex == '女') \
     .write \
     .parquet(r'users/gnd=female'
              , mode='overwrite')

In [6]:
# parquetから男性のみ読み込む
users_male = spark.read.parquet(r'users\sex=male')
users_male.groupby('sex').count().show()

+---+-----+
|sex|count|
+---+-----+
| 男|  927|
+---+-----+



In [8]:
# パーティションを結合してparquet全体を読み込む
users_all = spark.read.parquet('users')
users_all.show()
"""
※パーティションを結合した場合パーティションキー（この例では`sex`）と
　同じ名前のカラムがあると値が上書きされてしまったので本番では注意が必要
"""

+-------+-----------+-------------+---+---+----------+-------+----------+------+
|user_id|family_name|personal_name|age|sex|birth_date|pref_cd|active_flg|   gnd|
+-------+-----------+-------------+---+---+----------+-------+----------+------+
|   3844|       舟橋|         志歩| 35| 女| 1984/4/21|     17|         1|female|
|   8945|       茂木|         佳乃| 58| 女| 1961/9/16|     39|         1|female|
|   9465|       清家|           鈴| 53| 女|  1966/8/1|     31|         0|female|
|   5371|       植野|         麗子| 40| 女|  1980/2/9|     29|         1|female|
|   3868|       伊波|         利恵| 49| 女| 1970/6/26|     30|         0|female|
|   3725|       室井|           緑| 14| 女| 2006/2/22|     22|         0|female|
|   6801|       磯田|         栄美| 19| 女| 2000/7/12|     24|         1|female|
|   4614|       島袋|         栄美| 20| 女|  2000/1/3|     28|         0|female|
|   4983|       兼田|         奏音| 42| 女|1977/11/30|     33|         1|female|
|   2956|       塩原|       亜紀子| 37| 女|  1982/8/3|      7|         0|fema

'\n※パーティションを結合した場合パーティションキー（この例では`sex`）と\n\u3000同じ名前のカラムがあると値が上書きされてしまったので本番では注意が必要\n'